Import lib


In [1]:
import pandas as pd 
import numpy as np

Read data

In [2]:
df = pd.read_csv('../data.csv')

/var/folders/l9/lgcrjc856bb1g071byrjw6rr0000gn/T/ipykernel_70658/3454644713.py:1: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


In [3]:
columns_to_remove = [
    'id', 
    'member_id', 
    'title',
    'desc',
    'emp_title',
    'mths_since_last_delinq',
    'mths_since_last_record', 
    'mths_since_last_major_derog', 
    'tot_coll_amt', 
    'tot_cur_bal', 
    'open_acc_6m', 
    'open_il_6m', 
    'open_il_12m', 
    'open_il_24m', 
    'mths_since_rcnt_il', 
    'total_bal_il', 
    'il_util', 
    'open_rv_12m' ,
    'open_rv_24m', 
    'max_bal_bc', 
    'all_util', 
    'total_rev_hi_lim', 
    'inq_fi', 
    'total_cu_tl', 
    'inq_last_12m',
    'policy_code'
    ]
    


In [4]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_col

### Some criteria

### Important feature
- loan_amnt
- int_rate
- grade
- annual_inc
- purpose
- installments
- term

### Loan amount

In [5]:
df['loan_amnt'].unique()

array([ 5000,  2500,  2400, ..., 32825, 32575, 34175])

### Interest rate

In [6]:
df['int_rate'].unique()

array([10.65, 15.27, 15.96, 13.49, 12.69,  7.9 , 18.64, 21.28, 14.65,
        9.91, 16.29,  6.03, 11.71, 12.42, 14.27, 16.77,  7.51,  8.9 ,
       18.25,  6.62, 19.91, 17.27, 17.58, 21.67, 19.42, 22.06, 20.89,
       20.3 , 23.91, 19.03, 23.52, 23.13, 22.74, 22.35, 24.11,  6.  ,
       22.11,  7.49, 11.99,  5.99, 10.99,  9.99, 18.79, 11.49,  8.49,
       15.99, 16.49,  6.99, 12.99, 15.23, 14.79,  5.42, 10.59, 17.49,
       15.62, 21.36, 19.29, 13.99, 18.39, 16.89, 17.99, 20.62, 20.99,
       22.85, 19.69, 20.25, 23.22, 21.74, 22.48, 23.59, 12.62, 18.07,
       11.63,  7.91,  7.42, 11.14, 20.2 , 12.12, 19.39, 16.11, 17.54,
       22.64, 13.84, 16.59, 17.19, 12.87, 20.69,  9.67, 21.82, 19.79,
       18.49, 22.94, 24.59, 24.4 , 21.48, 14.82, 14.17,  7.29, 17.88,
       20.11, 16.02, 17.51, 13.43, 14.91, 13.06, 15.28, 15.65, 17.14,
       11.11, 10.37, 16.4 ,  7.66, 10.  , 18.62, 10.74,  5.79,  6.92,
        9.63, 14.54, 12.68, 19.36, 13.8 , 18.99, 21.59, 20.85, 21.22,
       19.74, 20.48,

### Grade

In [7]:
df['grade'].unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

### Purpose

In [8]:
df['purpose'].unique()

array(['credit_card', 'car', 'small_business', 'other', 'wedding',
       'debt_consolidation', 'home_improvement', 'major_purchase',
       'medical', 'moving', 'vacation', 'house', 'renewable_energy',
       'educational'], dtype=object)

### Installment

In [9]:
df['installment'].unique()

array([162.87,  59.83,  84.33, ..., 685.27, 185.06, 237.86])

### Term
- only 2 type: 36 & 60 months
- plan: binning

In [10]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

### Annual income 
- annual_inc_joint is total inc of all party involve
- Plan: aggregate 'annual_inc' and 'annual_inc_joint'
- How: if application_type is 'Joint' then replace 'annual_inc' with 'annual_inc_joint'

In [11]:
annual = df[df['application_type'] != 'INDIVIDUAL']

In [12]:
annual['annual_inc']

452290     29524.32
452477     86000.00
453477     89000.00
453733     60000.00
453789     78000.00
            ...    
548057     65000.00
548365      9000.00
548505    120000.00
549873     21559.00
550079     48000.00
Name: annual_inc, Length: 442, dtype: float64

In [13]:
annual['annual_inc_joint']

452290     94348.32
452477    182000.00
453477     89000.00
453733     95000.00
453789    111000.00
            ...    
548057     70000.00
548365     39000.00
548505    170000.00
549873     88596.00
550079     68000.00
Name: annual_inc_joint, Length: 442, dtype: float64

### Delinq_2yrs 

In [14]:
df['delinq_2yrs'].unique()

array([ 0,  2,  3,  1,  4,  6,  5,  8,  7,  9, 11, 15, 10, 12, 17, 18, 13,
       29, 24, 14, 22, 19, 16, 21, 30, 26, 20, 27, 39])

### DTI
- Also have individual and joint value => aggregate

In [15]:
df['dti']

0         27.65
1          1.00
2          8.72
3         20.00
4         17.94
          ...  
855964    28.69
855965     3.90
855966    30.90
855967    27.19
855968    10.83
Name: dti, Length: 855969, dtype: float64

In [16]:
dti = df[df['dti_joint'].isnull() == False]
dti['dti_joint']

452290    17.07
452477     6.44
453477    25.91
453733    23.99
453789    17.51
          ...  
548057    19.92
548365    18.65
548505    16.69
549873    15.40
550079    20.42
Name: dti_joint, Length: 442, dtype: float64

### Earliest credit line

In [17]:
df['earliest_cr_line'].unique()

array(['01-01-1985', '01-04-1999', '01-11-2001', '01-02-1996',
       '01-01-1996', '01-11-2004', '01-07-2005', '01-01-2007',
       '01-04-2004', '01-09-2004', '01-01-1998', '01-10-1989',
       '01-07-2003', '01-05-1991', '01-09-2007', '01-10-1998',
       '01-08-1993', '01-10-2003', '01-01-2001', '01-11-1997',
       '01-02-1983', '01-07-1985', '01-04-2003', '01-06-2001',
       '01-02-2002', '01-08-1984', '01-11-2006', '01-12-1987',
       '01-11-1981', '01-02-1997', '01-04-2005', '01-10-2007',
       '01-12-2000', '01-04-2007', '01-12-2001', '01-01-2003',
       '01-03-1994', '01-09-1998', '01-06-2004', '01-11-1995',
       '01-07-1999', '01-06-1995', '01-09-1992', '01-01-2002',
       '01-04-1992', '01-10-2006', '01-05-2000', '01-12-1998',
       '01-12-2004', '01-10-2000', '01-05-2002', '01-05-2006',
       '01-07-2002', '01-07-2006', '01-05-1997', '01-10-2005',
       '01-04-1995', '01-10-2002', '01-01-2000', '01-04-2000',
       '01-12-1994', '01-09-2005', '01-12-1984', '01-12

### Policy code
- Only 1 unique value -> not meaningful
- Consider drop

In [18]:
df['policy_code'].unique()

array([1])

### Issue date

In [19]:
df['issue_d'].unique()

array(['01-12-2011', '01-11-2011', '01-10-2011', '01-09-2011',
       '01-08-2011', '01-07-2011', '01-06-2011', '01-05-2011',
       '01-04-2011', '01-03-2011', '01-02-2011', '01-01-2011',
       '01-12-2010', '01-11-2010', '01-10-2010', '01-09-2010',
       '01-08-2010', '01-07-2010', '01-06-2010', '01-05-2010',
       '01-04-2010', '01-03-2010', '01-02-2010', '01-01-2010',
       '01-12-2009', '01-11-2009', '01-10-2009', '01-09-2009',
       '01-08-2009', '01-07-2009', '01-06-2009', '01-05-2009',
       '01-04-2009', '01-03-2009', '01-02-2009', '01-01-2009',
       '01-12-2008', '01-11-2008', '01-10-2008', '01-09-2008',
       '01-08-2008', '01-07-2008', '01-06-2008', '01-05-2008',
       '01-04-2008', '01-03-2008', '01-02-2008', '01-01-2008',
       '01-12-2007', '01-11-2007', '01-10-2007', '01-09-2007',
       '01-08-2007', '01-07-2007', '01-06-2007', '01-12-2013',
       '01-11-2013', '01-10-2013', '01-09-2013', '01-08-2013',
       '01-07-2013', '01-06-2013', '01-05-2013', '01-04

### Verification status
- 2 types: individual & joint -> aggregate

In [20]:
veri = df[df['verification_status_joint'].isnull() == False]
veri['verification_status']

452290           Verified
452477    Source Verified
453477           Verified
453733       Not Verified
453789    Source Verified
               ...       
548057    Source Verified
548365    Source Verified
548505    Source Verified
549873           Verified
550079           Verified
Name: verification_status, Length: 442, dtype: object

In [21]:
veri['verification_status_joint']

452290       Not Verified
452477       Not Verified
453477       Not Verified
453733       Not Verified
453789       Not Verified
               ...       
548057       Not Verified
548365    Source Verified
548505       Not Verified
549873           Verified
550079       Not Verified
Name: verification_status_joint, Length: 442, dtype: object

In [22]:
veri['application_type']

452290    JOINT
452477    JOINT
453477    JOINT
453733    JOINT
453789    JOINT
          ...  
548057    JOINT
548365    JOINT
548505    JOINT
549873    JOINT
550079    JOINT
Name: application_type, Length: 442, dtype: object

### Payment plan
- Binning 4sure

In [23]:
df['pymnt_plan'].unique()

array(['n', 'y'], dtype=object)